In [140]:
!pip3 install numpy nltk fastprogress

In [5]:
# python3.5 buildtagger.py <train_file_absolute_path> <model_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def train_model(train_file, model_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    train_file = sys.argv[1]
    model_file = sys.argv[2]
    start_time = datetime.datetime.now()
    train_model(train_file, model_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


Finished...
Time: 0:00:00.000121


In [5]:
# python3.5 runtagger.py <test_file_absolute_path> <model_file_absolute_path> <output_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def tag_sentence(test_file, model_file, out_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    test_file = sys.argv[1]
    model_file = sys.argv[2]
    out_file = sys.argv[3]
    start_time = datetime.datetime.now()
    tag_sentence(test_file, model_file, out_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


IndexError: list index out of range

## Creating the model statistics

Need to handle special cases like these:
 - fractions like: 1/2/CD
 - multiple listings: pianist/bassoonist/composer/NN
 
 As far as I understand all of the words belong to the same POS. 
 A function that adds both words should be implemented, but an initial version function should simply take the last two elements and uses them as word + tag.
 
 
 Also, likely the score could be improved by adding add-one-smoothing

In [132]:
import itertools
flatten = lambda x: list(itertools.chain.from_iterable(x))

class PosModelStats():
    def __init__(self, emissions, transitions, words, tags):
        self.emissions = emissions
        self.transitions = transitions
        self.words = words
        self.tags = tags
        

def extract_frequencies(filename):
    emission_dict = {}
    transition_dict = {}
    unique_words = set()
    unique_tags = set()

    with open(filename) as training_file:
        word_tag_pairs = [sentence.split(" ") for sentence in training_file.read().split("\n")]
        word_tag_pairs = flatten(word_tag_pairs)
        word_count = len(word_tag_pairs)
        previous_tag = word_tag_pairs[0].split("/")[-1]

        for word_tag in word_tag_pairs:
            if len(word_tag.split("/")) < 2:
                continue

            word, tag = word_tag.split("/")[-2:]

            unique_words.add(word)
            unique_tags.add(tag)

            if word_tag not in emission_dict.keys():
                emission_dict[word_tag] = 1
            else:
                emission_dict[word_tag] += 1

            if previous_tag + " " + tag not in transition_dict.keys():
                transition_dict[previous_tag + " " + tag] = 1
            else:
                transition_dict[previous_tag + " " + tag] += 1

            previous_tag = tag
    
    return PosModelStats(emission_dict, transition_dict, unique_words, unique_tags)
            
def add_one_smoothing():
    pass

In [133]:
%%time

pos_model_stats = extract_frequencies('sents.train')


TypeError: __init__() missing 1 required positional argument: 'common_tag'

In [71]:
import _pickle as pickle

def write_output(model_stats, filename="model-file"):
    with open(filename, "wb") as model_file:
        model_file.write(pickle.dumps(model_stats))

In [72]:
%%time

write_output(pos_model_stats)

CPU times: user 50.7 ms, sys: 8.89 ms, total: 59.6 ms
Wall time: 62.8 ms


In [110]:
with open("model-file", "rb") as model_read:
    loaded = pickle.load(model_read)

# Simple implementation of POS tagger

In [143]:
def read_and_tag(input_filename, pos_tagger):
    tagged_sent = []

    with open(input_filename, "r") as input_file:
        sentences = input_file.read().split("\n")

        for sentence in progress_bar(sentences):
            tagged_sent.append(pos_tagger(sentence))
    
    return tagged_sent

## Problems

How do I deal with words that have yet to be seen in the training data?

In [148]:
from functools import partial
from fastprogress import progress_bar

common_tag = "NN"

def word_emissions(word, keys):
    return [key for key in keys if key.startswith(word)]

def simple_pos_tagger(sentence, stats):
    tagged_sentence = []
    keys = stats.emissions.keys()
    
    for word in sentence.split(" "):
        emission_tags = word_emissions(word, keys)
        emission_probs = [stats.emissions[word_tag] for word_tag in emission_tags]

        if len(emission_tags) > 0:
            tagged_sentence.append(emission_tags[np.argmax(emission_probs)])
        else:
#             print("Word hasn't appeared in the training data: {}".format(word))
            tagged_sentence.append("{}/{}".format(word, common_tag))
        
    return " ".join(tagged_sentence)

simple_stats_tagger = partial(simple_pos_tagger, stats=loaded)

In [149]:
%%time

tagged = read_and_tag("sents.test", pos_tagger=simple_stats_tagger)

KeyboardInterrupt: 

In [136]:
tagged

['For six years , T. Marshall Hahn Jr. has made corporate acquisitions in the George Bush mode : kind and gentle .',
 'The question now : Can he act more like hard-charging Teddy Roosevelt ?',
 "Mr. Hahn , the 62-year-old chairman and chief executive officer of Georgia-Pacific Corp. is leading the forest-product concern 's unsolicited $ 3.19 billion bid for Great Northern Nekoosa Corp .",
 "Nekoosa has given the offer a public cold shoulder , a reaction Mr. Hahn has n't faced in his 18 earlier acquisitions , all of which were negotiated behind the scenes .",
 'So far , Mr. Hahn is trying to entice Nekoosa into negotiating a friendly surrender while talking tough .',
 "`` We are prepared to pursue aggressively completion of this transaction , '' he says .",
 'But a takeover battle opens up the possibility of a bidding war , with all that implies .',
 'If a competitor enters the game , for example , Mr. Hahn could face the dilemma of paying a premium for Nekoosa or seeing the company fal